In [1]:
from bs4 import BeautifulSoup as BS
import requests
import random
import traceback


go to (https://httpbin.org/ip) to see ur corrent ip address

import requests
from bs4 import BeautifulSoup as BS

def get_free_proxies():
    url = 'https://free-proxy-list.net/'  # Corrected URL
    response = requests.get(url)
    soup = BS(response.content, 'html.parser')
    
    # Check if the table exists before trying to access it
    proxy_table = soup.find("table", attrs={"class": "table table-striped table-bordered"})  # Updated selector
    
    if proxy_table is None:
        print("Could not find the proxy table. The website structure might have changed.")
        return []
        
    proxies = []
    # Only proceed if we found the table
    for row in proxy_table.find_all("tr")[1:]:
        tds = row.find_all("td")
        try:
            ip = tds[0].text.strip()
            port = tds[1].text.strip()
            proxies.append(str(ip) + ":" + str(port))
        except IndexError:
            continue

    return proxies

print(get_free_proxies())

url="https://httpbin.org/ip"
proxies=get_free_proxies()

for i in range(len(proxies)):
    print("Request Number : " + str(i+1))
    proxy=proxies[i]
    try:
        response=requests.get(url,proxies={"http":proxy})
        print(response.json())
    except:
        print("Not Available")
        
    

def scrape_with_proxy(target_url, proxy_list, retries=5):
    random.shuffle(proxy_list) # Shuffle the list to ensure randomness
    
    for i in range(retries):
        proxy = proxy_list[i % len(proxy_list)] # Cycle through the list
        proxies_dict = {"http": proxy, "https": proxy}
        print(f"Attempt {i+1}: Trying with proxy {proxy}")
        
        try:
            response = requests.get(target_url, proxies=proxies_dict, timeout=8)
            # Check if the request was successful
            if response.status_code == 200:
                print(f"Success! Scraped {target_url} with proxy {proxy}")
                return response
            else:
                print(f"Proxy {proxy} returned status code {response.status_code}. Retrying...")

        except requests.exceptions.RequestException as e:
            print(f"Proxy {proxy} failed to connect: {e}. Retrying...")

    print(f"Failed to scrape {target_url} after {retries} attempts.")
    return None # Return None if all retries fail

# --- Example Usage ---
all_proxies = get_free_proxies()
if all_proxies:
    target_site = "https://en.wikipedia.org/wiki/List_of_largest_companies_by_revenue" # A safe site for testing
    successful_response = scrape_with_proxy(target_site, all_proxies, retries=10)

    if successful_response:
        print("\n--- Snippet of the successfully scraped content: ---")
        print(successful_response.text[:300]) # Print the first 300 characters


In [2]:
URL='https://realpython.github.io/fake-jobs/'
headers={'User_Agent':'Mozilla/5.0(windows NT 10.0; Win64; x64'}
response=requests.get(URL,headers=headers)
response

<Response [200]>

In [3]:
from bs4 import BeautifulSoup as BS
soup=BS(response.text)

1.a. Use the .find method to find the tag containing the first job title ("Senior Python Developer"). Hint: can you find a tag type and/or a class that could be helpful for extracting this information? Extract the text from this title.  

In [4]:
soup.findAll('h2',class_='title is-5')[0].text.strip()

'Senior Python Developer'

1.b. Now, use what you did for the first title, but extract the job title for all jobs on this page. Store the results in a list.

In [5]:
titles=soup.findAll('h2',class_='title is-5')
title=[x.text.strip() for x in titles]

1.c. Finally, extract the companies, locations, and posting dates for each job. For example, the first job has a company of "Payne, Roberts and Davis", a location of "Stewartbury, AA", and a posting date of "2021-04-08". Ensure that the text that you extract is clean, meaning no extra spaces or other characters at the beginning or end

In [6]:
companies=soup.findAll('h3',class_='subtitle is-6 company')
company=[x.text.strip() for x in companies]

In [7]:
locations=soup.findAll('p',class_='location')
location=[x.text.strip() for x in locations]

In [8]:
posting_dates=soup.findAll('time',datetime='2021-04-08')
posting_date=[x.text.strip() for x in posting_dates]

In [9]:
combined_list=list(zip(title,company,location,posting_date))

In [10]:
import pandas as pd
df = pd.DataFrame(combined_list,columns=['title', 'company', 'location', 'posting_date'])
df

,title,company,location,posting_date
0,Senior Python Developer,"Payne, Roberts and Davis","Stewartbury, AA",2021-04-08
1,Energy engineer,Vasquez-Davidson,"Christopherville, AA",2021-04-08
2,Legal executive,"Jackson, Chambers and Levy","Port Ericaburgh, AA",2021-04-08
3,Fitness centre manager,Savage-Bradley,"East Seanview, AP",2021-04-08
4,Product manager,Ramirez Inc,"North Jamieview, AP",2021-04-08
...,...,...,...,...
95,Museum/gallery exhibitions officer,"Nguyen, Yoder and Petty","Lake Abigail, AE",2021-04-08
96,"Radiographer, diagnostic",Holder LLC,"Jacobshire, AP",2021-04-08
97,Database administrator,Yates-Ferguson,"Port Susan, AE",2021-04-08
98,Furniture designer,Ortega-Lawrence,"North Tiffany, AA",2021-04-08


2.a Next, add a column that contains the url for the "Apply" button.First, use the BeautifulSoup find_all method to extract the urls.  

In [11]:
urls=soup.findAll('a',class_='card-footer-item')
url=[x['href'] for x in urls]
url=url[1::2]
df['url']=url
df

,title,company,location,posting_date,url
0,Senior Python Developer,"Payne, Roberts and Davis","Stewartbury, AA",2021-04-08,https://realpython.github.io/fake-jobs/jobs/se...
1,Energy engineer,Vasquez-Davidson,"Christopherville, AA",2021-04-08,https://realpython.github.io/fake-jobs/jobs/en...
2,Legal executive,"Jackson, Chambers and Levy","Port Ericaburgh, AA",2021-04-08,https://realpython.github.io/fake-jobs/jobs/le...
3,Fitness centre manager,Savage-Bradley,"East Seanview, AP",2021-04-08,https://realpython.github.io/fake-jobs/jobs/fi...
4,Product manager,Ramirez Inc,"North Jamieview, AP",2021-04-08,https://realpython.github.io/fake-jobs/jobs/pr...
...,...,...,...,...,...
95,Museum/gallery exhibitions officer,"Nguyen, Yoder and Petty","Lake Abigail, AE",2021-04-08,https://realpython.github.io/fake-jobs/jobs/mu...
96,"Radiographer, diagnostic",Holder LLC,"Jacobshire, AP",2021-04-08,https://realpython.github.io/fake-jobs/jobs/ra...
97,Database administrator,Yates-Ferguson,"Port Susan, AE",2021-04-08,https://realpython.github.io/fake-jobs/jobs/da...
98,Furniture designer,Ortega-Lawrence,"North Tiffany, AA",2021-04-08,https://realpython.github.io/fake-jobs/jobs/fu...


2.b. Next, get those same urls in a different way. Examine the urls and see if you can spot the pattern of how they are constructed. Then, build the url using the elements you have already extracted. Ensure that the urls that you created match those that you extracted using BeautifulSoup. Warning: You will need to do some string cleaning and prep in constructing the urls this way. For example, look carefully at the urls for the "Software Engineer (Python)" job and the "Scientist, research (maths)" job.

In [12]:
base_url = 'https://realpython.github.io/fake-jobs/'
df['url1'] = (base_url + df['title'].str.lower().str.replace(' ', '-') + '-' + df.index.astype(str)+ '.html')

3. Finally, we want to get the job description text for each job.  
    a. Start by looking at the page for the first job, https://realpython.github.io/fake-jobs/jobs/senior-python-developer-0.html. Using BeautifulSoup, extract the job description paragraph. 

In [13]:
url2='https://realpython.github.io/fake-jobs/jobs/senior-python-developer-0.html'
headers={'User_Agent':'Mozilla/5.0(windows NT 10.0; Win64; x64'}
response=requests.get(url2,headers=headers)

In [14]:
soup=BS(response.text)
soup.findAll('p')[1].text

'Professional asset web application environmentally friendly detail-oriented asset. Coordinate educational dashboard agile employ growth opportunity. Company programs CSS explore role. Html educational grit web application. Oversea SCRUM talented support. Web Application fast-growing communities inclusive programs job CSS. Css discussions growth opportunity explore open-minded oversee. Css Python environmentally friendly collaborate inclusive role. Django no experience oversee dashboard environmentally friendly willing to learn programs. Programs open-minded programs asset.'

3b. We want to be able to do this for all pages. Write a function which takes as input a url and returns the description text on that page. For example, if you input "https://realpython.github.io/fake-jobs/jobs/television-floor-manager-8.html" into your function, it should return the string "At be than always different American address. Former claim chance prevent why measure too. Almost before some military outside baby interview. Face top individual win suddenly. Parent do ten after those scientist. Medical effort assume teacher wall. Significant his himself clearly very. Expert stop area along individual. Three own bank recognize special good along.".

In [15]:
def print_description(url):
    try:
        response=requests.get(url)
        soup=BS(response.text)
        paragraph=soup.findAll('p')[1].text
        return paragraph
    except:
        return ""
   

Q3.c c. Use the .apply method on the url column you created above to retrieve the description text for all of the jobs.

In [16]:
df['description']=df['url'].apply(print_description)   
 

BONUS
Q1. Navigate to https://www.billboard.com/charts/hot-100/


In [17]:
URL='https://www.billboard.com/charts/hot-100/'
headers=headers={'User_Agent':'Mozilla/5.0(windows NT 10.0; Win64; x64'}
response=requests.get(URL,headers=headers)
soup=BS(response.content,'html.parser')

Using BeautifulSoup, extract out the This Week's artist

In [21]:
artists= soup.select('span.c-label.a-no-trucate')
artist=[x.text.strip() for x in artists]
artist

['Taylor Swift',
 'HUNTR/X: EJAE, Audrey Nuna & REI AMI',
 'Alex Warren',
 'Olivia Dean',
 'Taylor Swift',
 'Leon Thomas',
 'Justin Bieber',
 'Mariah Carey',
 'Kehlani',
 'Morgan Wallen',
 'Wham!',
 'sombr',
 'Brenda Lee',
 'Bobby Helms',
 'Morgan Wallen Featuring Tate McRae',
 'Ella Langley',
 'Chris Brown Featuring Bryson Tiller',
 'Olivia Dean',
 'Ariana Grande',
 'Sabrina Carpenter',
 'Saja Boys: Andrew Choi, Neckwav, Danny Chung, Kevin Woo & samUIL Lee',
 'Sabrina Carpenter',
 'Ravyn Lenae',
 'Saja Boys: Andrew Choi, Neckwav, Danny Chung, Kevin Woo & samUIL Lee',
 'Cody Johnson',
 'Tate McRae',
 'Kelly Clarkson',
 'Taylor Swift',
 'Nat "King" Cole',
 'Andy Williams',
 'KATSEYE',
 'Michael Buble',
 'Riley Green Featuring Ella Langley',
 'Dean Martin',
 'NF',
 'HUNTR/X: EJAE, Audrey Nuna & REI AMI',
 'The Ronettes',
 'Megan Moroney',
 'Lainey Wilson',
 'Burl Ives',
 'Disco Lines & Tinashe',
 'HUNTR/X: EJAE, Audrey Nuna & REI AMI',
 'Russell Dickerson',
 'Taylor Swift',
 'Gunna Featu

extract this week's song.

In [22]:
songs=soup.select('h3.c-title.a-font-basic.u-letter-spacing-0010')
song=[x.text.strip() for x in songs]
song

['The Fate Of Ophelia',
 'Golden',
 'Ordinary',
 'Man I Need',
 'Opalite',
 'Mutt',
 'Daisies',
 'All I Want For Christmas Is You',
 'Folded',
 'I Got Better',
 'Last Christmas',
 'Back To Friends',
 "Rockin' Around The Christmas Tree",
 'Jingle Bell Rock',
 'What I Want',
 "Choosin' Texas",
 'It Depends',
 'So Easy (To Fall In Love)',
 'Santa Tell Me',
 'Manchild',
 'Soda Pop',
 'Tears',
 'Love Me Not',
 'Your Idol',
 "Travelin' Soldier",
 'Tit For Tat',
 'Underneath The Tree',
 'Elizabeth Taylor',
 'The Christmas Song (Merry Christmas To You)',
 "It's The Most Wonderful Time Of The Year",
 'Gabriela',
 "It's Beginning To Look A Lot Like Christmas",
 "Don't Mind If I Do",
 'Let It Snow! Let It Snow! Let It Snow!',
 'Fear',
 "How It's Done",
 'Sleigh Ride',
 '6 Months Later',
 'Somewhere Over Laredo',
 'A Holly Jolly Christmas',
 'No Broke Boys',
 'What It Sounds Like',
 'Happen To Me',
 'Wi$h Li$t',
 'wgft',
 'Father Figure',
 'Where Is My Husband!',
 'Christmas (Baby Please Come Home

extract song's peak position

peak_positions=soup.select('span.c-label.u-font-family-secondary\\@mobile-max.u-font-family-basic\\@tablet')
peak_positions=soup.select('li.lrv-u-flex.lrv-u-flex-shrink-0')